# Set up

In [87]:
import glob # for system ls
from natsort import natsorted # for alphabetical sorting

import xarray as xr # for reading netcdf
# import numpy as np
import dask.array as da
import dask
import rioxarray as rio # for writing tif

import os

In [83]:
# these are the same for everyone on HPC Orion

data_nc = '/work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/'
data_npy = '/work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/'
data_tif = '/work/hpc/datasets/un_fao/pyaez/global_1980/daily/tif/'
data_static = '/work/hpc/datasets/un_fao/pyaez/static/netcdf/'

varnames = ['Precip','Rhum','Srad','Tmax-2m','Tmin-2m','Wind-2m']
varnames365 = ['Precip365','Rhum365','Srad365','Tmax-2m365','Tmin-2m365','Wind-2m365']

# Main Code

# netcdf to npy

In [17]:
# each 3D array has total size ~12.7GB
# we need to chunk these arrays so that a chunk fits into memory (~9GB per single core)
# any chunk size less than ~9GB should work, we'll use 8 chunks --> ~1.5GB per chunk

chunks={'time':-1,'lat':450,'lon':2160} # 8 chunks

# look at size of the chunks
test=xr.open_dataset(data_nc+'Tmin-2m_daily_1980_5m.nc',chunks=chunks)['Tmin-2m'].sel(lat=slice(90,-60.)).transpose('lat','lon','time').data
test

dask.array<transpose, shape=(1800, 4320, 366), dtype=float32, chunksize=(450, 2160, 366), chunktype=numpy.ndarray>

In [7]:
del test

# do the nc to npy conversion for each variable
# we are also subsetting the global array (to eliminate artarctica where all grids are nan)
# expect about 1 min run time per variable

for var in varnames:
    # get nc file name
    f = natsorted(glob.glob(data_nc+var+'*_5m.nc'))[0]
        
    if f:
        # read netcdf data into a dask array of numpy array chunks
        print('reading',f)
        data = xr.open_dataset(f,chunks=chunks)[var].sel(lat=slice(90,-60.)).transpose('lat','lon','time').data        

        # set up dir for writing npy
        out_dir=data_npy+var+'/'
        isExist = os.path.exists(out_dir)
        if not isExist:
            os.makedirs(out_dir)
        # write npy data
        print('writing to',out_dir+'0.npy')     
        da.to_npy_stack(out_dir,data,axis=2)          
    else:
        print('no file',f)

reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Precip_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Precip/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Rhum_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Rhum/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Srad_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Srad/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Tmax-2m_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Tmax-2m/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Tmin-2m_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Tmin-2m/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Wind-2m_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Wind-2m/0.npy


In [88]:
# # to output files without the leap day run this

# # do the nc to npy conversion for each variable
# # we are also subsetting the global array (to eliminate artarctica where all grids are nan)
# # we also drop the leap day to output 365 total days
# # expect about 1 min run time per variable

# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
#     for var_in,var_out in zip(varnames,varnames365):
#         # get nc file name
#         f = natsorted(glob.glob(data_nc+var_in+'*_5m.nc'))[0]

#         if f:
#             # read netcdf data into a dask array of numpy array chunks
#             print('reading',f)
#             data = xr.open_dataset(f,chunks=chunks)[var_in].sel(lat=slice(90,-60.)).drop_sel(time='1980-02-29').transpose('lat','lon','time').data

#             # set up dir for writing npy
#             out_dir=data_npy+var_out+'/'
#             isExist = os.path.exists(out_dir)
#             if not isExist:
#                 os.makedirs(out_dir)
#             # write npy data
#             print('writing to',out_dir+'0.npy')     
#             da.to_npy_stack(out_dir,data,axis=2)          
#         else:
#             print('no file',f)

reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Precip_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Precip365/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Rhum_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Rhum365/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Srad_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Srad365/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Tmax-2m_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Tmax-2m365/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Tmin-2m_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy/Tmin-2m365/0.npy
reading /work/hpc/datasets/un_fao/pyaez/global_1980/daily/netcdf/Wind-2m_daily_1980_5m.nc
writing to /work/hpc/datasets/un_fao/pyaez/global_1980/daily/npy

# netcdf mask and elevation to tif

### mask

In [46]:
# get file path and file name
f = glob.glob(data_static+'mask_*_5m.nc')[0]
filename=f.split('/')[-1]
print(f)
print(filename)

/work/hpc/datasets/un_fao/pyaez/static/netcdf/mask_2268708_5m.nc
mask_2268708_5m.nc


In [51]:
print('reading',f)

# load mask from netcdf file
data = xr.open_dataset(f)['mask'].sel(lat=slice(90,-60.))

# write file
outfile='/work/hpc/datasets/un_fao/pyaez/static/tif/'+filename[:-3]+'.tif'
print('writing',outfile)
data.rio.to_raster(outfile)

reading /work/hpc/datasets/un_fao/pyaez/static/netcdf/mask_2268708_5m.nc
writing /work/hpc/datasets/un_fao/pyaez/static/tif/mask_2268708_5m.tif


### elevation

In [53]:
# get file path and file name
f = glob.glob(data_static+'Elevation_*_5m.nc')[0]
filename=f.split('/')[-1]
print(f)
print(filename)

/work/hpc/datasets/un_fao/pyaez/static/netcdf/Elevation_2268708_5m.nc
Elevation_2268708_5m.nc


In [57]:
print('reading',f)

# load mask from netcdf file
data = xr.open_dataset(f)['Elevation'].sel(lat=slice(90,-60.))

# write file
outfile='/work/hpc/datasets/un_fao/pyaez/static/tif/'+filename[:-3]+'.tif'
print('writing',outfile)
data.rio.to_raster(outfile)

reading /work/hpc/datasets/un_fao/pyaez/static/netcdf/Elevation_2268708_5m.nc
writing /work/hpc/datasets/un_fao/pyaez/static/tif/Elevation_2268708_5m.tif
